# Gaussian Graphical Model on a 4‑Cycle
This notebook simulates a 4‑dimensional Gaussian random vector whose precision matrix corresponds to the cyclic graph 1–2–3–4–1.
We verify conditional independence structure and estimate covariance/precision matrices.

-We consider the following precision matrix 


$$
\begin{pmatrix}
10 & 5 & 0 &3\\
5 & 10 & 5 &0\\
0 & 5 & 10 &5\\
3 & 0 & 5 & 10\\
\end{pmatrix}.
$$

## 1. Define the graph and precision matrix

In [9]:
import numpy as np
import pandas as pd

d = 4
Theta = np.array([
    [10,  5,  0,  3],
    [ 5, 10,  5,  0],
    [ 0,  5, 10,  5],
    [ 3,  0,  5, 10]
])
Sigma = np.linalg.inv(Theta)
Sigma

array([[ 0.29411765, -0.26470588,  0.23529412, -0.20588235],
       [-0.26470588,  0.38823529, -0.31176471,  0.23529412],
       [ 0.23529412, -0.31176471,  0.38823529, -0.26470588],
       [-0.20588235,  0.23529412, -0.26470588,  0.29411765]])

## 2. Simulate Gaussian sample

We simulate a large sample from the multivariate normal distribution with covariance matrix $\Sigma = \Theta^{-1}$

In [10]:
N = 10**6
mean = np.zeros(d)
X = np.random.multivariate_normal(mean, Sigma, size=N)
X[:5]

array([[ 0.45807124, -0.33909788,  0.58144707, -0.70324022],
       [ 0.10057896,  0.07218014,  0.19295134,  0.16697862],
       [-0.0913466 ,  0.89910805, -0.32482867,  0.43996053],
       [-0.24195046,  0.47259962, -0.83097377,  0.20010532],
       [ 0.32726198, -0.54604308,  1.25448694, -0.40347625]])

## 4. Estimate covariance and precision

In [11]:
EstimSigma = np.cov(X, rowvar=False)
EstimTheta = np.linalg.inv(EstimSigma)
print("This is the estimated precision matrix obtained by first estimating the sample covariance matrix and then inverting it:\n")
print(EstimTheta)


This is the estimated precision matrix obtained by first estimating the sample covariance matrix and then inverting it:

[[ 1.00068077e+01  4.99899934e+00 -1.42978274e-03  3.00381078e+00]
 [ 4.99899934e+00  9.99757935e+00  5.00950823e+00  1.69910246e-02]
 [-1.42978274e-03  5.00950823e+00  1.00301190e+01  5.02326262e+00]
 [ 3.00381078e+00  1.69910246e-02  5.02326262e+00  1.00098996e+01]]


## 5. Maximum-likelihood estimation

We parameterize $\Theta$ using only the **upper triangular entries**. We then implement the log-Likelihood function:

$$
L(\Theta) =  \log \det(\Theta) - trace (S \cdot \Theta), 
$$
where $S$ is the MLE covariance matrix.


In [13]:
from scipy.optimize import minimize

def construct_symmetric_matrix(theta_vec):
    Theta_mat = np.zeros((d, d))
    idx = np.triu_indices(d)
    Theta_mat[idx] = theta_vec
    Theta_mat = Theta_mat + Theta_mat.T - np.diag(np.diag(Theta_mat))
    return Theta_mat

def log_likelihood(theta_vec, S):
    Theta_mat = construct_symmetric_matrix(theta_vec)
    det = np.linalg.det(Theta_mat)
    if det <= 0:
        return -np.inf
    return np.log(det) - np.trace(S @ Theta_mat)



A = np.random.randn(d, d)
Theta0 = A @ A.T + d * np.eye(d)
theta0_vec = Theta0[np.triu_indices(d)]




res = minimize(lambda t: -log_likelihood(t, EstimSigma),
               theta0_vec)

Theta_MLE = construct_symmetric_matrix(res.x)
print("This is the estimated precision matrix obtained by maximizing the log-likelihood function:\n")
print(Theta_MLE)


This is the estimated precision matrix obtained by maximizing the log-likelihood function:

[[ 1.00066346e+01  4.99900257e+00 -1.20809375e-03  3.00391926e+00]
 [ 4.99900257e+00  9.99744528e+00  5.00968909e+00  1.73645957e-02]
 [-1.20809375e-03  5.00968909e+00  1.00301148e+01  5.02325542e+00]
 [ 3.00391926e+00  1.73645957e-02  5.02325542e+00  1.00095328e+01]]


C:\Users\20254817\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


## 6. Conclusion

- We simulated a Gaussian vector respecting a graph structure  
- We estimated covariance and precision matrices  
- We computed the **MLE precision matrix** and compare it with the inverse of the sample coavriance matrix 
- The MLE recovers the original sparsity pattern of the graph

This provides a full demonstration of how Gaussian graphical models behave in practice.
